# Data Science und zufriedene Kunden (in R)

Heute stellen wir uns der Frage, ob und wie gut ein Unternehmen abschätzen kann, welche Kunden es demnächst verlieren könnte. Ist das angefertigte Modell gut genug, kann versucht werden, Maßnahmen zu implementieren, die dafür sorgen, dass der Kunde gehalten wird.

Ich nehme schon einmal vorweg, dass wir uns dem Problem mittels *Deep Learning* bzw. einem *Artificial Neural Network* (kurz: *ANN*) nähern werden. Hier soll dabei die Theorie nur am Rande betrachtet werden, sondern viel mehr der Fokus auf der Benutzung der Server von *H2O* liegen.

Was ist dieses *H2O*? Eine Firma mit dem Ziel, künstliche Intelligenz für jedermann verfügbar zu machen. So zumindest das selbsterklärte Firmenziel. Für uns ist an dieser Stelle nur eines relevant: ANNs können sehr rechenintensiv sein... und H2O gestattet uns, ihre Server zu nutzen. Wer also keine high-end Grafikkarte hat: weiterlesen!

## Datensatz
Unser Datensatz heute kommt von einer Bank (natürlich anonymisiert) und ist insofern etwas Besonderes, als dass ich mit ihm selbst das erste Mal *Deep Learning* betrieben habe. Ihr findet die Daten wieder auf Kaggle.com unter dem Namen *Churn_Modelling.csv*.

Zunächst lesen wir einmal die Daten ein und schauen uns die ersten 10 Zeilen an:

In [6]:
# Importing the dataset
dataset = read.csv('Churn_Modelling.csv')
dataset[1:10,]

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


Die ersten drei Spalten enthalten die Zeilennummer, die jeweilige Kunden-ID und den Nachnamen des Kunden - also die Informationen, die benötigt werden, um den Kunden zu identifizieren. Oder anders formuliert die Daten, die für unsere Analyse aller Wahrscheinlichkeit keine Bedeutung haben.

Also: Raus damit!

In [7]:
# Adjusting the dataset
dataset = dataset[4:14]

Jetzt wird es schon interessanter: *Credit Score*, Heimatland (*Geography*), Geschlecht usw.. In der letzten Spalte kommt dann das, weshalb wir uns die Daten überhaupt anschauen - der aktuelle Kundenstatus. "1" für ehemalige Kunden, "0" für Bestandskunden.

Bis auf das Heimatland und das Geschlecht liegt alles auch schon in numerischer Form vor. Passen wir also noch schnell diese zwei Spalten an, Stichwort *Encoding*. In R verwenden wir hierfür die *factor*-Funktion, mit der wir einem Vektor aus Levels einen Vektor aus Labeln zuordnen können. In unserem Fall wollen wir natürlich Zahlen als Label für unsere Daten haben. Um das korrekte Format zu bekommen, schieben wir dem Ganzen dann noch die Funktion *as.numeric* vor:

In [8]:
# Enconding the categorical variables as factors (using 'as.numeric()' for deep learning package)
dataset$Geography = as.numeric(factor(dataset$Geography,
                                      levels = c('France', 'Spain', 'Germany'),
                                      labels = c(1, 2, 3)))
dataset$Gender = as.numeric(factor(dataset$Gender,
                           levels = c('Male', 'Female'),
                           labels = c(1, 2)))
dataset[1:5,]

CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,1,2,42,2,0.00,1,1,1,101348.88,1
608,2,2,41,1,83807.86,1,0,1,112542.58,0
502,1,2,42,8,159660.80,3,1,0,113931.57,1
699,1,2,39,1,0.00,2,0,0,93826.63,0
850,2,2,43,2,125510.82,1,1,1,79084.10,0


Nun teilen wir unser Datenset wie gewohnt in ein Trainings- und ein Testset. Hierbei hilft uns die *caTools*-Bibliothek. Zum Installieren verwendet man einfach den *install.packages*-Befehl, wie unten zu sehen. Ich habe die Bibliothek schon installiert, also kommentiere ich den Befehl aus und lade nur noch die Bibliothek via *library(caTools)*. Aus den Daten definieren wir dann 75-90% der Daten als Trainingsset und den Rest als Testset. Hier wähle ich ein Verhältnis von 80:20 von Trainings- zu Testdaten.

Schließlich müssen wir noch *Feature Scaling* betreiben. Das bedeutet, dass wir die Werte, die unsere unabhängigen Variablen annehmen können, auf eine gleichmäßige Reichweite beschränken. Mathematisch gesehen ist dies für neuronale Netzwerke nicht nötig, aber ohne *Feature Scaling* würde das Training des Netzwerks ewig dauern. Hier hilft uns der *scale*-Befehl. Beachtet, dass wir unsere abhängige Variable nicht skalieren wollen - sie liegt ohnehin schon bei "0" oder "1". Aus diesem Grund ignorieren wir die 11. Spalte.

In [9]:
# install.packages('caTools')
library(caTools)
set.seed(123)
split = sample.split(dataset$Exited, SplitRatio = 0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

# Feature Scaling
training_set[-11] = scale(training_set[-11])
test_set[-11] = scale(test_set[-11])

training_set[1:5,]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,-0.32756459,-0.8895031,1.0929950,0.2962052,-1.0313943,-1.2190517,-0.9111120,0.6518863,0.970619,0.0216855,1
3,-1.53810353,-0.8895031,1.0929950,0.2962052,1.0432370,1.3330389,2.5466186,0.6518863,-1.030142,0.2400424,1
5,2.06247381,0.3202937,1.0929950,0.3914787,-1.0313943,0.7871702,-0.9111120,0.6518863,0.970619,-0.3646920,0
6,-0.05855594,0.3202937,-0.9148029,0.4867522,1.0432370,0.5992723,0.8177533,0.6518863,-1.030142,0.8617430,1
7,1.77277219,-0.8895031,-0.9148029,1.0583930,0.6974651,-1.2190517,0.8177533,0.6518863,0.970619,-1.5624706,0


## Modell - Deep Learning

Nun kommen wir zum spannenden Teil - unser künstliches neuronales Netzwerk. Wir erinnern uns, wie das Schema hierfür aussehen soll: Das Netzwerk besteht aus mehreren Schichten (Layern), welche wiederum aus Neuronen zusammengesetzt sind. Das erste (Input-)Layer wird aus unseren Input-Variablen gebildet, das letzte, sogenannte Output-Layer hingegen beschreibt, ob das Netzwerk denkt, dass ein Kunde das Unternehmen verlassen wird. Dazwischen liegen die versteckten oder zu englisch *hidden* Layer, die wir über mehrere Zyklen (Epochen) trainieren werden.

Nun aber zur Benutzung von H2O. Zunächst muss wieder die entsprechende Bibliothek installiert sein. Dieses Mal ist sie auch etwas größer, aber es lohnt sich! Nachdem wir die Bibliothek dann geladen haben, stellen wir eine Verbindung mit den H2O-Servern her. Hierfür nutzen wir das Kommando *h2o.init()*. Zusätzlich geben wir hier eine Option hinzu, die die Anzahl an verwendeten Rechenkernen bestimmt: *nthreads*. Da wir keinen bestimmten Server definiert haben (was man aber durchaus könnte), wissen wir nicht genau, wieviele Kerne uns zur Verfügung stehen. Wählt man aber den Wert "-1", werden schlicht alle Kerne verwendet. Und genau dies wollen wir.

Schließlich erstellen wir unser Deep-Learning-Modell mittels *h2o.deeplearning*. Wichtig ist hier, dass unser Trainingsset als h2o-Objekt geladen wird. Dies erreichen wir mittels der Funktion *as.h2o*. Für die Aktivierungsfunktion empfehle ich die *Rectifier*-Funktion. (Der interessierte Leser wird hier online schnell fündig.) Für die hidden Layer definieren wir anschließend deren Anzahl, sowie die Menge an Neuronen pro Layer. Was hier die beste Konfiguration ist, lässt sich nicht genau sagen, aber als Faustregel sollte man sich am Mittelwert von Input- und Output-Neuronen orientieren. In unserem Fall bedeutet dies mit 10 Input-Neuronen + 1 Output-Neuron, dass wir gerundet etwa 6 Layer mit jeweils 6 Neuronen verwenden sollten. Schließlich definieren wir die Anzahl an Epochen, die das Netzwerk lernen soll, und wie häufig dabei die Gewichte (*weights*) zwischen den Neuronen aktulisiert werden. Letzteres wird mit der Option *train_samples_per_iteration* festgelegt, wobei der Wert "-2" eine automatisches Tuning veranlasst und uns die Arbeit abnimmt. And that's it!

In [10]:
# install.packages('h2o')
library(h2o)
h2o.init(nthreads = -1)
classifier = h2o.deeplearning(y = 'Exited',
                              training_frame = as.h2o(training_set),
                              activation = 'Rectifier',
                              hidden = c(6,6),
                              epochs = 100,
                              train_samples_per_iteration = -2)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         26 minutes 38 seconds 
    H2O cluster timezone:       Europe/Berlin 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    3 months and 11 days !!! 
    H2O cluster name:           H2O_started_from_R_george_qif010 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.92 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.6.1 (2019-07-05) 


Warning message in h2o.clusterInfo():
“
Your H2O cluster version is too old (3 months and 11 days)!
Please download and install the latest version from http://h2o.ai/download/”


  |======================================================================| 100%
  |======================================================================| 100%


Ging doch ziemlich schnell! Glaubt mir, auf unseren Heim-PCs wäre das eine ganz andere Hausnummer gewesen.

Nun nutzen wir unseren classifier und wenden ihn auf unser Testset an. Zur Erinnerung: Im Trainingsset gab es zwei Typen von Kunden - die ehemaligen (Wert 1), und die Noch-Kunden (Wert 0). Unser Output-Neuron ist allerdings nicht binär, sondern liegt zwischen im Intervall von 0 bis 1. Je höher der Wert, desto wahrscheinlicher ist es, dass der Kunde bald das Unternehmen verlassen wird. Zum Abgleich mit dem tatsächlichen Kunden-Status, transformieren wir das Intervall (0 bis 1) wieder ins Binäre, wobei Werte größer als 0,5 auf 1 gesetzt werden, und alle anderen auf 0.

In [11]:
# Predicting the Test set results
prop_pred = h2o.predict(classifier, newdata = as.h2o(test_set[-11]))
y_pred = (prop_pred > 0.5)
y_pred = as.vector(y_pred)

  |======================================================================| 100%
  |======================================================================| 100%


## Auswertung

Die Stunde der Wahrheit. Wie gut hat unser Modell abgeschnitten? Wir befragen die *Confusion Matrix*. Zur Erinnerung: Die Zeilen geben den realen Wert an, die Spalten den geschätzten. Entsprechend sind die Einträge auf der Diagonalen die korrekten und die auf der Antidiagonalen die falschen Prognosen. Die Summe der Diagonaleinträge geteilt durch die Summe aller Einträge liefert uns dann die Genauigkeit (*accuracy*) unseres Modells.

In [12]:
# Making the Confusion Matrix
cm = table(test_set[, 11], y_pred)
cm
(cm[1,1]+cm[2,2])/2000

   y_pred
       0    1
  0 1532   61
  1  211  196

[1] 0.864

86%! Das ist deutlich besser als blindes Raten mit 50% Trefferquote. Allerdings geht das noch deutlich besser, was an dieser Stelle aber nicht ausgeführt werden soll. Nun beenden wir noch die Verbindung zum H2O-Server und freuen uns schon auf das nächste Mal, wenn wir H2O verwenden. Tolles Tool!

In [13]:
h2o.shutdown()

Are you sure you want to shutdown the H2O instance running at http://localhost:54321/ (Y/N)?  Y


[1] TRUE